In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

In [2]:
water_train1=pd.read_csv("./Watermelon-train1.csv",encoding='gb2312')
labels_origin=['色泽','根蒂','敲声','纹理','好瓜']

water_train1=water_train1[labels_origin]
water_label=water_train1["好瓜"]
print(water_train1)
watertest1=pd.read_csv("./Watermelon-test1.csv",encoding='gb2312')
label_test=labels_origin[:]
watertest1=watertest1[label_test]
#
# print(watertest1)

    色泽  根蒂  敲声  纹理 好瓜
0   青绿  蜷缩  浊响  清晰  是
1   乌黑  蜷缩  沉闷  清晰  是
2   乌黑  蜷缩  浊响  清晰  是
3   青绿  蜷缩  沉闷  清晰  是
4   浅白  蜷缩  浊响  清晰  是
5   青绿  稍蜷  浊响  清晰  是
6   乌黑  稍蜷  浊响  稍糊  是
7   乌黑  稍蜷  浊响  清晰  是
8   乌黑  稍蜷  沉闷  稍糊  否
9   青绿  硬挺  清脆  清晰  否
10  浅白  硬挺  清脆  模糊  否
11  浅白  蜷缩  浊响  模糊  否
12  青绿  稍蜷  浊响  稍糊  否
13  浅白  稍蜷  沉闷  稍糊  否
14  浅白  蜷缩  浊响  模糊  否
15  青绿  蜷缩  沉闷  稍糊  否


In [3]:
water_train2=pd.read_csv("./Watermelon-train2.csv",encoding='gb2312')
watertest2=pd.read_csv("./Watermelon-test2.csv",encoding='gb2312')
labels_origin2=['色泽','根蒂','敲声','纹理','密度','好瓜']
watertest2=watertest2[labels_origin2]
water_train2=water_train2[labels_origin2]
print(watertest2)

   色泽  根蒂  敲声  纹理     密度 好瓜
0  乌黑  稍蜷  浊响  清晰  0.403  是
1  青绿  稍蜷  浊响  稍糊  0.481  是
2  乌黑  稍蜷  浊响  清晰  0.337  是
3  乌黑  稍蜷  沉闷  稍糊  0.666  否
4  青绿  硬挺  清脆  清晰  0.243  否


In [4]:
from collections import Counter
import math
def entropy(dataset):
    label_values = dataset[dataset.columns[-1]]
    counts =  label_values.value_counts()
    Ent = 0
    for c in label_values.unique():
        freq = counts[c]/len(label_values) 
        Ent =Ent- freq*math.log(freq,2)
    return Ent

In [5]:
def ration(group,length_data):
    ratio=len(group)/length_data
    return (ratio)*entropy(group)
def gain_col(dataset,col):
    ent_gross=entropy(dataset)
    #print(dataset.groupby(col))
    data_grounp=dataset.groupby(col)
    ent_sum=0
    for name,item in data_grounp:
        ent_sum+=ration(item,len(dataset))
    return ent_gross-ent_sum



In [6]:
def max_col(dataset):
    max_info_gain=0
    labels=dataset.columns[:-1]
    #print(labels)
    max_label=None
    for col in labels:
        gain=gain_col(dataset,col)
        if gain>max_info_gain:
            max_info_gain=gain
            max_label=col
    return max_label

In [7]:
def decisiontree_fit(dataset):
    data=dataset[labels_origin]
    label=dataset.iloc[:,-1]
def get_type(dataset,loc):
    return  len(Counter([element for element in dataset[loc]]))
def get_most_label(labels):
    return (labels.value_counts()).idxmax()
def get_mid_rep(dataset,col):
    to_get=sorted(dataset[col])
    lis=[]
    for item in np.arange(len(to_get)-1):
        lis.append((to_get[item]+to_get[item+1])/2)
    return lis
    

In [8]:
def decision_tree(dataset,label_list):
    feature = dataset.columns[:-1]
    label_list = dataset.iloc[:, -1]
    if len(feature)==0 or get_type(dataset,feature)==1:
        return get_most_label(label_list)
    elif len(pd.unique(label_list)) == 1:
        return label_list.values[0]
    bestAttr = max_col(dataset)
    tree = {bestAttr: {}}
    for attri_value,tree_data in dataset.groupby(by=bestAttr):
        if len(tree_data) == 0:
            tree[bestAttr][attri_value] = get_most_label(label_list)
        else:
            new_data = tree_data.drop(bestAttr,axis=1)
            tree[bestAttr][attri_value] = decision_tree(new_data,water_label)
    return tree
decision_tree = decision_tree(water_train1,water_label)
print(decision_tree)


{'纹理': {'模糊': '否', '清晰': {'根蒂': {'硬挺': '否', '稍蜷': '是', '蜷缩': '是'}}, '稍糊': {'色泽': {'乌黑': {'敲声': {'沉闷': '否', '浊响': '是'}}, '浅白': '否', '青绿': '否'}}}}


In [9]:
def predict(tree, data):
    feature = list(tree.keys())[0]
    #print(feature)
    label = data[feature]
    next_tree = tree[feature][label]
    if type(next_tree) == str:
        return next_tree
    else:
        return predict(next_tree, data)
print(watertest1.columns)
labels=watertest1['好瓜']
acc=0
for num in np.arange(len(watertest1)):
    if predict(decision_tree ,watertest1.loc[num,:])==labels[num]:
        acc+=1
print('Accuracy: %f'%(acc/len(watertest1)))

Index(['色泽', '根蒂', '敲声', '纹理', '好瓜'], dtype='object')
Accuracy: 0.700000


In [10]:
labels=[0,0,0,0,1,0]
label_dict={}
for index, label in zip(water_train2, labels):
    label_dict[index] = label
#print(label_dict)

针对连续数的使用，将采取以下方式：


In [11]:


def type_attri(dataset,attr):
    if dataset[attr].dtype == 'float64':
        return 1
    return 0
def discrete(dataset,attr):
    ent_parent=entropy(dataset)
    attribution = dataset[attr].unique()
    entD = 0
    for attri in attribution:
        value = dataset[dataset[attr]== attri]
        entD += len(value)/len(dataset)*entropy(value)
    return (entropy(dataset)/ent_parent - entD/ent_parent,None)

def contin(dataset,attr,mid_term):
    entD = entropy(dataset)
    gains = []
    for item in mid_term:
        data_posi = dataset[dataset[attr] > item]
        data_neg = dataset[dataset[attr] <= item]
        sum_Ent=ration(data_posi,len(data_posi))+ration(data_neg,len(data_neg))
        gain_ratio = (entD - sum_Ent)/entD
        gains.append((gain_ratio,item))
    return max(gains)
def gain(dataset,attr):
    if type_attri(dataset,attr):
        split_p = get_mid_rep(dataset,attr)
        return contin(dataset,attr,split_p)
    else:
        return discrete(dataset,attr)
def takeSecond(elem):
    return elem[1]
def split(dataset):
    attrs = dataset.columns[:-1]
    gain_list=[]
    for item in attrs:
        gain_list.append((item,gain(dataset,item)))
    #print(gain_list)
    gain_list.sort(key = takeSecond,reverse = True)
    return gain_list[0]
def generateTree(data,label_list):
    label_values =  data.iloc[:, -1]
    if len(label_values.unique()) == 1:
        return label_values.values[0]
    if data.shape[1] == 1 or (data.shape[1]==2 and len(data.T.ix[0].unique())==1):
        return get_most_label(label_values)
    bestAttr,(gain_attri,t_data) = split(data)
    if t_data is None:
        DecesionTree = {bestAttr:{}}
        values = data[bestAttr].unique() 
        for val in values:
            attrsAndLabel = data.columns.tolist()
            data_cur = data[data[bestAttr]== val]
            attrsAndLabel.remove(bestAttr)
            data_cur  = data_cur [attrsAndLabel]
            DecesionTree[bestAttr][val] = generateTree(data_cur ,label_list)
        return DecesionTree
    else:
        node = bestAttr+'<'+str(t_data)
        DecesionTree = {node:{}}
        values = ['是','否']
        for val in values:
            if val == '是':
                data_cur  = data[data[bestAttr] <= t_data] 
            elif val=="否": 
                data_cur =data[data[bestAttr] > t_data]
            DecesionTree[node][val] = generateTree(data_cur ,label_list)
        return DecesionTree

In [12]:
tree= generateTree (water_train2,water_train2.columns)
print(tree)

{'纹理': {'清晰': {'密度<0.3815': {'是': '否', '否': '是'}}, '稍糊': {'密度<0.56': {'是': '是', '否': '否'}}, '模糊': '否'}}
